Scraping inspirations for finn.no:

-[python](https://github.com/qiangwennorge/ScrapeFinnBolig)

-[node.js-scraper](https://github.com/Lekesoldat/finn-scraper)


Example of bussiness property is [this building](https://www.finn.no/realestate/businesssale/ad.html?finnkode=335466271). 

It can be scraped using [BeautifulSoup](https://www.crummy.com/software/BeautifulSoup/bs4/doc).

In [2]:
import requests
from bs4 import BeautifulSoup
# for splitting strings
import re
from osgeo import ogr, osr

For now we want to focus on Sarpsborg. By adding a paramter to the URL, we can narrow our search for the listings in Sarpsborg

In [3]:
mainurl = 'https://www.finn.no/realestate/businesssale/search.html?location='
# Sarpsborg location
sarpsborg = '1.20002.20023'

In [4]:
def RequestAndScrape(url) :
    r = requests.get(url)
    if r.status_code == 200 :
        soup = BeautifulSoup(r.content, 'html.parser')
        return soup
    else :
        print('Bad Request:', r.status_code)

In [5]:
def FetchListings(soup) :
    arr_listings = []
    # find the listings in articles with the following class
    listings = soup.find_all('a', {'class':'sf-search-ad-link'} )
    i = 0
    for listing in listings : 
        # get the value of the url
        listing_url = listing['href']
        arr_listings.append(listing_url)
        i += 1
    # count listings
    print(i)
    return arr_listings

In [6]:
url = mainurl + sarpsborg
soup = RequestAndScrape(url)

listings = FetchListings(soup)


22


Per listing, I want the following information: 

    
    - Title of listing
    - Adress of listing
    - FINN-kode
    - Type of listing
    - Area information
    - Real Estate Agent


In [17]:
addresses = []
for listing in listings : 
    listing_data = RequestAndScrape(listing)
    # title of listing
    title = listing_data.find('h1').text
    
    # adress is streetname number, postcode city
    address = listing_data.find('span', {'data-testid':'object-address'}).text
    addresses.append(address)

None


Nominatim API that is used, see [this link](https://nominatim.org/release-docs/develop/api/Search/)

In [16]:
# iterate over list of addresses
for full_address in addresses :
    # geocode adress
    adress = re.split(', | ', full_address )
    street  = adress[0] + ' ' + adress[1]
    postalcode = adress[2]
    city = adress[3]

    geocode_url = 'https://nominatim.openstreetmap.org/search?'
    params = dict (
        adressdetails = '1',
        extratags = '1',
        namedetails = '0',
        limit = '1',
        polygon_geojson= '1',
        format = 'geojson',
        street = street,
        city = city,
        postalcode = postalcode
    )

    r = requests.get(geocode_url, params=params)
    if r.status_code == 200 :
        features = r.json()['features']
    
        if len(features) > 0 :
            feature = features[0]

            osm_type = feature['properties']['type']
            osm_diplayname = feature['properties']['display_name']

            geometry = ogr.CreateGeometryFromJson(str(feature['geometry']))
            source = geometry.GetSpatialReference()   
            # prints the EPSGCode of layer
            print('EPSG-code of geometry: ', source.GetAttrValue('AUTHORITY', 1))

            

    else : 
        print('The following request failed:')
        print(r.url)
        print('With status code, ', r.status_code)

GEOGCS["WGS 84",
    DATUM["WGS_1984",
        SPHEROID["WGS 84",6378137,298.257223563,
            AUTHORITY["EPSG","7030"]],
        AUTHORITY["EPSG","6326"]],
    PRIMEM["Greenwich",0,
        AUTHORITY["EPSG","8901"]],
    UNIT["degree",0.0174532925199433,
        AUTHORITY["EPSG","9122"]],
    AXIS["Latitude",NORTH],
    AXIS["Longitude",EAST],
    AUTHORITY["EPSG","4326"]]
EPSG


KeyboardInterrupt: 